# Setup

Import a few common modules

In [39]:
#CODE for points = 1
# import sklearn, numpy, os
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >="0.20"
print("sklearn version is: ", sklearn.__version__ )


# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard


# to make this notebook's output stable across runs
np.random.seed(42)


sklearn version is:  0.22.2.post1
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [40]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [41]:
z = np.linspace(-5, 5, 200)
logit(z)

array([0.00669285, 0.00703534, 0.00739523, 0.00777338, 0.00817071,
       0.00858818, 0.00902677, 0.00948756, 0.00997163, 0.01048013,
       0.01101428, 0.01157533, 0.01216461, 0.01278351, 0.01343346,
       0.01411598, 0.01483266, 0.01558515, 0.01637519, 0.01720457,
       0.01807518, 0.018989  , 0.01994809, 0.02095457, 0.0220107 ,
       0.02311881, 0.02428131, 0.02550075, 0.02677974, 0.02812103,
       0.02952746, 0.03100199, 0.03254769, 0.03416773, 0.03586542,
       0.03764418, 0.03950753, 0.04145915, 0.04350281, 0.04564241,
       0.04788198, 0.05022565, 0.0526777 , 0.05524249, 0.05792451,
       0.06072839, 0.06365881, 0.06672059, 0.06991865, 0.07325795,
       0.07674359, 0.08038068, 0.08417443, 0.08813008, 0.09225289,
       0.09654816, 0.10102115, 0.10567713, 0.11052132, 0.11555887,
       0.12079485, 0.12623421, 0.13188177, 0.13774216, 0.14381982,
       0.15011896, 0.15664352, 0.16339714, 0.1703831 , 0.17760434,
       0.18506337, 0.19276223, 0.20070249, 0.2088852 , 0.21731

## Xavier and He Initialization

In [42]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

### Leaky ReLU

In [43]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [44]:
#CODE for points = 1
#load MNIST dataset from keras

from keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [45]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [46]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [47]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 7s 4ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200


Now look at what happens if we try to use the ReLU activation function instead:

In [48]:
np.random.seed(42)
tf.random.set_seed(42)

In [49]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [50]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [52]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 29s 17ms/step - loss: 0.8247 - accuracy: 0.6724 - val_loss: 0.7727 - val_accuracy: 0.6852
Epoch 2/5
1719/1719 [==============================] - 29s 17ms/step - loss: 0.6985 - accuracy: 0.7256 - val_loss: 0.6522 - val_accuracy: 0.7342
Epoch 3/5
1719/1719 [==============================] - 29s 17ms/step - loss: 0.6719 - accuracy: 0.7450 - val_loss: 0.6275 - val_accuracy: 0.7754
Epoch 4/5
1719/1719 [==============================] - 29s 17ms/step - loss: 0.6113 - accuracy: 0.7727 - val_loss: 0.6243 - val_accuracy: 0.7720
Epoch 5/5
1719/1719 [==============================] - 29s 17ms/step - loss: 0.6670 - accuracy: 0.7539 - val_loss: 0.5930 - val_accuracy: 0.7930


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [54]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [55]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 784)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_232 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_10 (Bat  (None, 300)              1200      
 chNormalization)                                                
                                                                 
 dense_233 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_11 (Bat  (None, 100)            

In [56]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization_9/gamma:0', True),
 ('batch_normalization_9/beta:0', True),
 ('batch_normalization_9/moving_mean:0', False),
 ('batch_normalization_9/moving_variance:0', False)]

In [57]:
#bn1.updates #deprecated

In [58]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [59]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.8491 - accuracy: 0.7130 - val_loss: 0.5606 - val_accuracy: 0.8086
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5753 - accuracy: 0.8007 - val_loss: 0.4785 - val_accuracy: 0.8368
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5182 - accuracy: 0.8194 - val_loss: 0.4432 - val_accuracy: 0.8466
Epoch 4/5
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4800 - accuracy: 0.8332 - val_loss: 0.4211 - val_accuracy: 0.8526
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4546 - accuracy: 0.8405 - val_loss: 0.4058 - val_accuracy: 0.8592


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [60]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [61]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [62]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 6ms/step - loss: 1.0640 - accuracy: 0.6715 - val_loss: 0.6875 - val_accuracy: 0.7836
Epoch 2/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6903 - accuracy: 0.7787 - val_loss: 0.5609 - val_accuracy: 0.8174
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6036 - accuracy: 0.8014 - val_loss: 0.5030 - val_accuracy: 0.8372
Epoch 4/5
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5514 - accuracy: 0.8164 - val_loss: 0.4674 - val_accuracy: 0.8448
Epoch 5/5
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5163 - accuracy: 0.8260 - val_loss: 0.4431 - val_accuracy: 0.8520


# Faster Optimizers

## Momentum optimization

In [63]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate= 0.01, momentum= 0.9)

## Nesterov Accelerated Gradient

In [65]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum= 0.9, nesterov=0.1)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [66]:
#CODE for points = 0.5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l1(0.01))

In [67]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 15s 8ms/step - loss: 1.5647 - accuracy: 0.7783 - val_loss: 1.0067 - val_accuracy: 0.7666
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.9059 - accuracy: 0.7952 - val_loss: 0.8494 - val_accuracy: 0.8216


In [68]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [69]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 16s 9ms/step - loss: 1.6248 - accuracy: 0.7795 - val_loss: 1.0283 - val_accuracy: 0.7626
Epoch 2/2
1719/1719 [==============================] - 15s 8ms/step - loss: 0.9096 - accuracy: 0.7933 - val_loss: 0.8523 - val_accuracy: 0.8188
